In [5]:
import requests
import json
import pandas as pd
import re
from bs4 import BeautifulSoup

In [4]:
home_page_url="https://www.lenskart.com/"
session = requests.session()
response=session.get(home_page_url)
response

<Response [200]>

In [27]:
category_url = "https://api-gateway.juno.lenskart.com/v2/products/category/3363?page-size=1000&page=0"

headers = {
    "Content-Type": "application/json",
    "Referer": "https://www.lenskart.com/",
    "Sec-Ch-Ua": '"Chromium";v="124", "Microsoft Edge";v="124", "Not-A.Brand";v="99"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": '"Windows"',
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36 Edg/124.0.0.0",
    "X-Accept-Language": "en",
    "X-Api-Client": "desktop",
    "X-B3-Traceid": "991714373712786",
    "X-Country-Code": "IN",
    "X-Country-Code-Override": "IN"
}

session = requests.session()
response = session.get(category_url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Parse JSON response
    data = response.json()

    # Write JSON data to a file
    with open("product_data.json", "w") as file:
        json.dump(data, file)
        
    print("Data has been successfully written to product_data.json")
else:
    print("Failed to retrieve data. Status code:", response.status_code)


Data has been successfully written to product_data.json


In [30]:
# Load JSON data from file
with open("product_data.json", "r") as file:
    data = json.load(file)

# Extract necessary information and flatten the structure
product_list = data["result"]["product_list"]
products = []

for product in product_list:
    product_id = product["id"]
    image_url = product["image_url"]
    product_url = product["product_url"]
    color = product["color"]
    size = product.get("size", None)  # Handle missing key
    width = product.get("width", None)  # Handle missing key
    brand_name = product["brand_name"]
    model_name = product["model_name"]
    market_price = next((price["price"] for price in product["prices"] if price["name"] == "Market Price"), None)
    lenskart_price = next((price["price"] for price in product["prices"] if price["name"] == "Lenskart Price"), None)
    wishlist_count = product["wishlistCount"]
    purchase_count = product["purchaseCount"]
    avg_rating = product["avgRating"]
    total_no_of_ratings = product.get("totalNoOfRatings", None)  # Handle missing key
    
    products.append({
        "product_id": product_id,
        "image_url": image_url,
        "product_url": product_url,
        "color": color,
        "size": size,
        "width": width,
        "brand_name": brand_name,
        "model_name": model_name,
        "market_price": market_price,
        "lenskart_price": lenskart_price,
        "wishlist_count": wishlist_count,
        "purchase_count": purchase_count,
        "avg_rating": avg_rating,
        "total_no_of_ratings": total_no_of_ratings
    })

# Create DataFrame
df = pd.DataFrame(products)

# Convert DataFrame to CSV
df.to_csv("products.csv", index=False)

print("CSV file saved successfully.")


CSV file saved successfully.


In [31]:
# List of category IDs
category_ids = [3363, 8427, 8415, 3362]

# Base URL and headers
base_url = "https://api-gateway.juno.lenskart.com/v2/products/category/"
headers = {
    "Content-Type": "application/json",
    "Referer": "https://www.lenskart.com/",
    "Sec-Ch-Ua": '"Chromium";v="124", "Microsoft Edge";v="124", "Not-A.Brand";v="99"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": '"Windows"',
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36 Edg/124.0.0.0",
    "X-Accept-Language": "en",
    "X-Api-Client": "desktop",
    "X-B3-Traceid": "991714373712786",
    "X-Country-Code": "IN",
    "X-Country-Code-Override": "IN"
}

# Create an empty list to store all product data
all_products = []

# Iterate through category IDs
for category_id in category_ids:
    # Construct category URL
    category_url = f"{base_url}{category_id}?page-size=1000&page=0"
    
    # Send request
    session = requests.session()
    response = session.get(category_url, headers=headers)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse JSON response
        data = response.json()
        products = data["result"]["product_list"]
        
        # Extend the list of products
        all_products.extend(products)
        print(f"Data for category {category_id} retrieved successfully.")
    else:
        print(f"Failed to retrieve data for category {category_id}. Status code:", response.status_code)

# Write all products to a JSON file
with open("all_product_data.json", "w") as file:
    json.dump(all_products, file)
    
print("All data has been successfully written to all_product_data.json.")

# Convert JSON data to DataFrame
df = pd.DataFrame(all_products)

# Convert DataFrame to CSV
df.to_csv("all_products.csv", index=False)

print("CSV file saved successfully.")


Data for category 3363 retrieved successfully.
Data for category 8427 retrieved successfully.
Data for category 8415 retrieved successfully.
Data for category 3362 retrieved successfully.
All data has been successfully written to all_product_data.json.
CSV file saved successfully.


In [40]:
#Scrap store data 
store_page_url="https://www.lenskart.com/stores"
session = requests.session()
response=session.get(store_page_url)
with open('stores.html','wb') as file:
    file.write(response.content)

In [41]:
# Read HTML file
with open("stores.html", "r", encoding="utf-8") as file:
    html_content = file.read()

# Define regex pattern to match "slug" attribute values
pattern = r'"slug":"(.*?)"'

# Search for "slug" attribute values using regex and store them in a list
slug_list = re.findall(pattern, html_content)

print(slug_list)


['lenskart-com-chhatarpur-mehrauli-new-delhi-136896', 'lenskart-com-jawahar-nagar-optometrists-jawahar-nagar-new-delhi-77003', 'optometrist-sunglasses-paschim-vihar-new-delhi-60858', 'optometrist-sunglasses-model-town-new-delhi-60859', 'optometrist-sunglasses-old-rajendar-nagar-new-delhi-60860', 'lenskart-com-geeta-colony-geeta-colony-delhi-183288', 'lenskart-com-vasundhra-enclave-vasundhara-enclave-new-delhi-420102', 'optometrist-sunglasses-dwarka-sector-6-new-delhi-60872', 'optometrist-sunglasses-vikaspuri-new-delhi-60873', 'optometrist-sunglasses-east-patel-nagar-new-delhi-60874', 'optometrist-sunglasses-naraina-vihar-new-delhi-60875', 'lenskart-com-sector-7-rohini-sector-7-rohini-new-delhi-838503', 'lenskart-com-uttam-nagar-uttam-nagar-new-delhi-551705', 'lenskart-com-amar-colony-amar-colony-new-delhi-96382', 'lenskart-com-sector-17-rohini-rohini-delhi-649291', 'lenskart-com-sda-market-sda-market-delhi-180064', 'optometrist-sunglasses-rohini-sector-9-new-delhi-60896', 'optometrist-

In [6]:
locations_list = ['andhra-pradesh', 'arunachal-pradesh', 'assam', 'bihar', 'chattisgarh', 'chennai', 'chhattisgarh', 'dadra-and-nagar-haveli', 'delhi', 'goa', 'gujarat', 'gwalior', 'haryana', 'himachal-pradesh', 'hyderabad', 'jammu-and-kashmir', 'jharkhand', 'karnataka', 'kerala', 'madhya-pradesh', 'maharashtra', 'manipur', 'nagaland', 'new-delhi', 'odisha', 'patna', 'puducherry', 'punjab', 'rajasthan', 'sikkim', 'tamil-nadu', 'tamil\xa0nadu', 'telangana', 'tripura', 'uttar-pradesh', 'uttarakhand', 'west-bengal'] 

session = requests.session()

# Initialize lists to store data
store_names = []
ratings = []
review_counts = []
addresses = []
hours_list = []

for location in locations_list:
    store_page_url = f"https://www.lenskart.com/stores/location/{location}"
    response = session.get(store_page_url)
    
    # Parse HTML
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract store details
    store_cards = soup.find_all('div', class_="StoreCard_storeCard__3lKXu")
    
    for card in store_cards:
        # Store Name
        store_name_tag = card.find('a', class_='StoreCard_name__mrTXJ')
        store_name = store_name_tag.text.strip() if store_name_tag else ''
        store_names.append(store_name)
        
        # Rating
        rating_tag = card.find('div', class_='StoreCard_rating__CPC2K')
        rating = rating_tag.text.strip() if rating_tag else ''
        ratings.append(rating)
        
        # Review Count
        review_count_tag = card.find('div', class_='StoreCard_reviewCount__FT2tY')
        review_count = review_count_tag.text.strip().replace('(', '').replace(')', '') if review_count_tag else ''
        review_counts.append(review_count)
        
        # Address
        address_tag = card.find('p', id='store-address')
        address = address_tag.find('a', class_='StoreCard_storeAddress__PfC_v').text.strip() if address_tag else ''
        address = re.sub('\s+', ' ', address)  # Replace multiple spaces with a single space
        addresses.append(address)

        # Hours 
        hours_tag = card.find('div', class_='StoreCard_storeAddress__PfC_v')
        hours = hours_tag.text.strip().replace('Hours: ', '') if hours_tag else ''
        hours = re.sub('\s+', ' ', hours)  # Replace multiple spaces with a single space
        hours_list.append(hours)

# Create DataFrame
data = {
    'Store Name': store_names,
    'Rating': ratings,
    'Review Count': review_counts,
    'Address': addresses,
    'Hours': hours_list,
}

df = pd.DataFrame(data)

# Display DataFrame
print(df.head())

# Convert DataFrame to CSV
df.to_csv("store_locations.csv", index=False)

print("CSV file saved successfully.")


      Store Name Rating Review Count  \
0   Vizianagaram    4.9         1423   
1       Ramnagar    4.9         6065   
2       Kakinada    4.9         1683   
3          Eluru    4.9          907   
4  Shanthi Nagar    4.9         2907   

                                             Address                 Hours  
0  20-20-103, Plot No103, Madhuvan Towers, Krishn...  11:00 AM to 09:00 PM  
1  Shop No 10/4/16/2, Upper Ground Floor, Uplands...  11:00 AM to 09:30 PM  
2  20-1-34, Subash Road, Opposite Super Bazar, Ka...  11:00 AM to 09:00 PM  
3  24B-1-46/1, Vijaya Vihar Centre, Vijaya Vihar ...  11:00 AM to 09:00 PM  
4  Shop No 23/7/131, Shanthi Nagar, Mr Palli Circ...  11:00 AM to 09:00 PM  
CSV file saved successfully.
